In [1]:
import gzip
import numpy as np
import time
from scipy.stats import pearsonr, spearmanr, rankdata, ks_2samp
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
!pwd

# Extract fragments that are nonzero in both reps or zero in both reps

In [2]:
signals = open('../../sureseq/SUREfragmentsignal3.txt')
header = signals.readline()

In [3]:
rep1Data = []
rep2Data = []
rep1GoodData = []
rep2GoodData = []
goodSignals = []

In [4]:
# Take a backwards-pass through the data, removing any lines not satisfying either:
# biorep1Signal[i] > 0 and biorep2Signal[i] > 0 or biorep1Signal[i] == 0 and biorep2Signal[i] == 0
i = 0
for line in signals:
    if (i-1) % 1e7 == 0:
        print "On fragment %s / " %(i-1) + '153000000'
    line = line.strip().split('\t')
    rep1Signal = float(line[4])
    rep2Signal = float(line[5])
    rep1Data.append(rep1Signal)
    rep2Data.append(rep2Signal)
    seqLen = int(line[2]) - int(line[1])
#     if ((rep1Signal > 0 and rep2Signal > 0) or (rep1Signal == 0 and rep2Signal == 0)) and seqLen <= 2000:
    if rep1Signal > 0 and rep2Signal > 0 and seqLen <= 2000:
        rep1GoodData.append(rep1Signal)
        rep2GoodData.append(rep2Signal)
        name = line[0] + ":" + line[1] + "-" + line[2] + '(' + line[3] + ')'
        avgSignal = float(line[6])
        goodSignals.append([line[0], line[1], line[2], name, avgSignal, line[3], rep1Signal, rep2Signal])
    i += 1

On fragment 0 / 153000000
On fragment 10000000 / 153000000
On fragment 20000000 / 153000000
On fragment 30000000 / 153000000
On fragment 40000000 / 153000000
On fragment 50000000 / 153000000
On fragment 60000000 / 153000000
On fragment 70000000 / 153000000
On fragment 80000000 / 153000000
On fragment 90000000 / 153000000
On fragment 100000000 / 153000000
On fragment 110000000 / 153000000
On fragment 120000000 / 153000000
On fragment 130000000 / 153000000
On fragment 140000000 / 153000000
On fragment 150000000 / 153000000


In [5]:
goodSignals = np.array(goodSignals)

In [6]:
print len(goodSignals)

2090287


In [7]:
print goodSignals[:5]

[['chr1' '10225' '10368' 'chr1:10225-10368(+)' '1.43046568444' '+'
  '2.11804651617' '1.07989853381']
 ['chr1' '10271' '10772' 'chr1:10271-10772(+)' '47.9206004287' '+'
  '31.7706977425' '56.1547237583']
 ['chr1' '10271' '11710' 'chr1:10271-11710(-)' '9.29802694885' '-'
  '10.5902325808' '8.63918827051']
 ['chr1' '10587' '11454' 'chr1:10587-11454(-)' '113.006789071' '-'
  '131.318884002' '103.670259246']
 ['chr1' '11261' '12065' 'chr1:11261-12065(-)' '1.07284926333' '-'
  '2.11804651617' '0.539949266907']]


In [8]:
header = 'chr\tstart\tend\tname\tavgSignal\tstrand\trep1Signal\trep2Signal\n'

np.savetxt(fname = '../data/positiveReplicateSignalsSizeSelected.bed',
           fmt = '%s',
           X = goodSignals,
           delimiter = '\t', 
           header = header,
           comments='# ')

In [ ]:
print pearsonr(rep1GoodData, rep2GoodData)
print spearmanr(rep1GoodData, rep2GoodData)

In [ ]:
cnt = 0
for i in range(len(rep1GoodData)):
    if rep1GoodData[i] > 0:
        cnt += 1

In [ ]:
print float(cnt) / len(rep1GoodData) * 100.0

In [ ]:
rep1Data = np.array(rep1Data)
rep2Data = np.array(rep2Data)
rep1GoodData = np.array(rep1GoodData)
rep1GoodData = np.array(rep1GoodData)

In [ ]:
L = len(rep1Data)

In [ ]:
print 100.0 * np.sum(rep1Data > 0) / L
print 100.0 * np.sum(rep2Data > 0) / L
print 100.0 * np.sum(np.logical_and(rep1Data == 0, rep2Data == 0)) / L
print 100.0 * np.sum(np.logical_and(rep1Data > 0, rep2Data == 0)) / L
print 100.0 * np.sum(np.logical_and(rep1Data == 0, rep2Data > 0)) / L
print 100.0 * np.sum(np.logical_and(rep1Data > 0, rep2Data > 0)) / L

In [ ]:
greaterThanZeroOrZero = np.logical_or(np.logical_and(rep1Data > 0, rep2Data > 0), np.logical_and(rep1Data == 0, rep2Data == 0))
print spearmanr(rep1Data[greaterThanZeroOrZero], rep2Data[greaterThanZeroOrZero])

# Create train+val+test splits, fragment labels, fragment features

In [2]:
import sys

runName = 'Jun24RegressionPositives'
# runName = 'Jun22ClassificationWithoutQc'
chrs = ['chr' + str(i) for i in range(1, 23)]
valChrs = ['chr18']
testChrs = ['chr8']
trainChrs = chrs
trainChrs = [chrom for chrom in trainChrs if chrom not in valChrs and chrom not in testChrs]
# print trainChrs

dataMatrix = open('../data/positiveReplicateSignalsSizeSelected.bed', 'r').readlines()[2:]
# print len(dataMatrix)
# sys.exit(0)
# header = dataMatrix.readline()
# spacer = dataMatrix.readline()

trainSplit = open('../splits/train' + runName + '.txt', 'w')
valSplit = open('../splits/val' + runName + '.txt', 'w')
testSplit = open('../splits/test' + runName + '.txt', 'w')

# runName = 'Jun22ClassificationWithoutQc'
# classificationLabels = open('../labels/labels' + runName + '.txt', 'w')
# classificationLabels.write('name\tavg\trep1\trep2\n')
# runName = 'Jun22RegressionWithoutQc'
regressionLabels = open('../labels/labels' + runName + '.txt', 'w')
regressionLabels.write('name\tavg\trep1\trep2\n')

counts = [0,0,0,0]
for line in dataMatrix:
    line = line.strip().split('\t')
    chrom = line[0]
    fragmentName = line[3]
    if chrom in chrs:
        counts[3] += 1
    
    # add fragment names to their respective split files
    if chrom in trainChrs:
        trainSplit.write(fragmentName + '\n')
        counts[0] += 1
    elif chrom in valChrs:
        valSplit.write(fragmentName + '\n')
        counts[1] += 1
    elif chrom in testChrs:
        testSplit.write(fragmentName + '\n')
        counts[2] += 1
    
    avgSignal = float(line[4])
    rep1Signal = float(line[6])
    rep2Signal = float(line[7])
        
    # if the fragment was in any of the relevant splits, then add it to the labels file
    if chrom in trainChrs or chrom in valChrs or chrom in testChrs:
#         classificationLabel = fragmentName + '\t' + str(int(avgSignal > 0)) + '\t' + \
#                               str(int(rep1Signal > 0)) + '\t' + str(int(rep2Signal > 0)) + '\n'
#         classificationLabels.write(classificationLabel)
        
        regressionLabel = fragmentName + '\t' + str(avgSignal) + '\t' + \
                          str(rep1Signal) + '\t' + str(rep2Signal) + '\n'
        regressionLabels.write(regressionLabel)

trainSplit.close()
valSplit.close()
testSplit.close()
regressionLabels.close()

print "train size = " + str(counts[0]) + " fragments"
print "validation size = " + str(counts[1]) + " fragments"
print "test size = " + str(counts[2]) + " fragments"
print "total data size = " + str(counts[3]) + " fragments"

train size = 1851920 fragments
validation size = 51838 fragments
test size = 102992 fragments
total data size = 2006750 fragments


In [ ]:
!gzip ../splits/*.txt
!gzip ../labels/*.txt

# BedTools getfasta command to extract sequences

In [ ]:
!bedtools getfasta -fi ../../../data/hg19.fa -bed ../data/positiveReplicateSignals.bed -s -fo ../features/sequencesJun24RegressionPositives.fa

# Pad fasta sequences after using bedtools

In [3]:
print "opening files..."

runName = 'Jun24RegressionPositives'
sequences = open('../features/sequences' + runName + '.fa', 'r')
paddedSequences = open('../features/paddedSequences' + runName + '.fa', 'w')

print "padding..."

PADDED_LENGTH = 2000
i = 0
for line in sequences:
    if i % 2 == 0:
        paddedSequences.write(line)
    else:
        seq = line.strip()
        length = len(seq)
        padLength = (PADDED_LENGTH - length) / 2
        newSeq = 'N' * padLength + seq + 'N' * padLength
        if len(newSeq) < PADDED_LENGTH:
            newSeq += 'N'
        paddedSequences.write(newSeq + '\n')
    i += 1

opening files...
padding...


In [ ]:
!pwd

# Check correctness of features and labels files

In [ ]:
import gzip

sequences = gzip.open("../features/paddedSequencesRegressionJun24Positives.fa.gz", 'r')
i = 0
for line in sequences:
    print len(line.strip())
    if i > 100:
        break